In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def generate_time_series(num_timeseries, n_steps):
    
    freq1, freq2, phase1, phase2 = np.random.rand(4, num_timeseries, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time-phase1) * (freq1 * 10 + 10)) # First sine wave
    series += 0.2 * np.sin((time-phase2) * (freq2 * 20 + 20)) # Second sine wave
    series += 0.1 * (np.random.rand(num_timeseries, n_steps) - 0.5) # Noise

    return series[..., np.newaxis].astype(np.float32)

In [3]:
# Let's generate 10000 time series of 50 steps in order to predict one step ahead.
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

In [4]:
# X_train contains 7000 time series of shape [50, 1], X_train is [7000, 50, 1]
# y_train takes the last element of each series as a label
X_train, y_train = series[:7000, :n_steps ], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [5]:
print(X_train.shape)
print(y_train.shape)

(7000, 50, 1)
(7000, 1)


# Simple RNN

In [6]:
# input_shape is [None, 1] because a RNN model can process sequences of any length, and this is univariate time series.
# Also we will use the simplest architecture: a single layer with 1 neuron.
simple_rnn = keras.Sequential([
    layers.SimpleRNN(1, input_shape=[None, 1])
])
simple_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


For each neuron, there is one parameter per input and per hidden step dimension (in this case it is the number of neurons in the layer), plus a bias term. That's just 3 parameters.

In [7]:
optimizer = keras.optimizers.Adam(lr=0.01)
simple_rnn.compile(loss="mse", optimizer=optimizer)

By default the SimpleRNN uses a tanh activation function. The initial state is set to 0 and it is passed to a single recurrent neuron along with the value of the first time step x(0). The neuron computes a weighted sum of these values, applies tanh to the result and gives the result to the new state that is passed again to the same recurrent neuron along with x(1) for processing repeatedly until last time step until the layer just outputs the last value y49. This is done simultaneously for every time series

In [8]:
# Our model will train on batches of 32 x 219x50x1. 
history = simple_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 3s 10ms/step - loss: 0.0187 - val_loss: 0.0110
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0114 - val_loss: 0.0110
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0112 - val_loss: 0.0111
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0112 - val_loss: 0.0111
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0113 - val_loss: 0.0110
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0113 - val_loss: 0.0112
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0113 - val_loss: 0.0111
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0110 - val_loss: 0.0111
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0113 - val_loss: 0.0111
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 0.0113 - val_loss: 0.011

# Deep RNN

In [9]:
deep_rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.Dense(1)
])

In [10]:
deep_rnn.compile(loss="mse", optimizer=optimizer)

In [11]:
history = deep_rnn.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 6s 20ms/step - loss: 0.1971 - val_loss: 0.1761
Epoch 2/20
219/219 [==============================] - 3s 16ms/step - loss: 0.1432 - val_loss: 0.1753
Epoch 3/20
219/219 [==============================] - 3s 15ms/step - loss: 0.1488 - val_loss: 0.1674
Epoch 4/20
219/219 [==============================] - 3s 15ms/step - loss: 0.1479 - val_loss: 0.1537
Epoch 5/20
219/219 [==============================] - 3s 15ms/step - loss: 0.1486 - val_loss: 0.1496
Epoch 6/20
219/219 [==============================] - 4s 16ms/step - loss: 0.1519 - val_loss: 0.1475
Epoch 7/20
219/219 [==============================] - 3s 15ms/step - loss: 0.1470 - val_loss: 0.1581
Epoch 8/20
219/219 [==============================] - 5s 24ms/step - loss: 0.1470 - val_loss: 0.1473
Epoch 9/20
219/219 [==============================] - 4s 19ms/step - loss: 0.1421 - val_loss: 0.1487
Epoch 10/20
219/219 [==============================] - 3s 14ms/step - loss: 0.1437 - val_lo

In [16]:
series[7000, -10:, 0]

array([ 0.5164187 ,  0.40241325,  0.23317498, -0.03226947, -0.2284828 ,
       -0.35232672, -0.35885486, -0.27879146, -0.265251  , -0.3167747 ],
      dtype=float32)